## Code to take list of election candidates and create excel file from this

In [1]:
# Import needed libraries
import pandas as pd
from tabula import read_pdf

In [31]:
# Create a list of dataframes for each page
#filename = 'primarycandidatelist2022'
filename = 'generalcandidatelist2022'

# Read in PDF and create a list of dataframes for each page
df = read_pdf(f'../data/raw/candidate-information/{filename}.pdf',
              pages='all')

In [32]:
for n in range(0, len(df)):
    
    if n==0:
        df_all = df[n]
        
    else:
        df_all = pd.concat([df_all, df[n]],
                                   ignore_index=True)

In [33]:
# Fill in all the race names
office_col = 'For the Office Of...'
df_all[office_col] = df_all[office_col].fillna(method
                                                            ='ffill')

In [34]:
# Fix issue with unnamed columns
df_all['Address'] = df_all['Address'].fillna(df_all['Unnamed: 0'])
df_all['Phone'] = df_all['Phone'].fillna(df_all['Unnamed: 1'])
df_all['Email'] = df_all['Email'].fillna(df_all['Unnamed: 2'])
df_all['Filing Date'] = df_all['Filing Date'].fillna(df_all['Unnamed: 3'])

In [35]:
# Remove unanamed columns
df_all = df_all.drop(['Unnamed: 0',
                      'Unnamed: 1',
                      'Unnamed: 2',
                      'Unnamed: 3'], axis=1)

# Drop address and filing date
df_all = df_all.drop(['Address', 'Filing Date'], axis=1)

# Remove blank/filler lines
df_all = df_all.dropna(axis=0, subset=['Ballot Name(s)'])

# Get only races with a candidate
df_all = df_all.loc[df_all['Ballot Name(s)'] != 'No Candidate']

In [36]:
# Reorder the columns
df_all = df_all.reindex([office_col,
                                         'Ballot Name(s)',
                                         'Party',
                                         'Email',
                                         'Phone'], axis=1)

In [37]:
# Remove races we are not asking
df_all = df_all[df_all[office_col].str
                                .contains('Auditor of State') == False]
df_all = df_all[df_all[office_col].str
                            .contains('Treasurer of State') == False]
df_all = df_all[df_all[office_col].str
                                .contains('Attorney General') == False]

In [38]:
df_all

,For the Office Of...,Ballot Name(s),Party,Email,Phone
0,United States Senator,Chuck Grassley,Republican,Candidate did not provide,319-983-2458
1,United States Senator,Michael Franken,Democratic,mtf@frankenforiowa.org,712-635-8356
2,United States Representative District 1,Mariannette Miller-Meeks,Republican,info@drmillermeeks.com,641-226-0528
3,United States Representative District 1,Christina Bohannan,Democratic,christinab@bohannanforcongress.com,319-321-5823
4,United States Representative District 2,Ashley Hinson,Republican,info@ashleyhinson.com,319-804-8395
...,...,...,...,...,...
307,State Representative District 98,Monica Kurth,Democratic,mkurth10@gmail.com,563-271-9332
308,State Representative District 99,Matthew Rinker,Republican,mrinker.burl@gmail.com,319-750-5156
309,State Representative District 99,Dennis M. Cohoon,Democratic,dennis.cohoon@yahoo.com,319-759-2168
310,State Representative District 100,Martin L. Graber,Republican,col_graber@yahoo.com,319-470-5522


In [47]:
# Figure out which races are contested
df_all['Contested'] = 'yes'

# Loop through each unique offices
for office in df_all[office_col].unique():
    
    df_office = df_all[df_all[office_col] == office]
    
    # Check number in each party running
    if 'primary' in filename:
        num_running = df_office.pivot_table(columns=['Party'],
                                            aggfunc='size')

    # What counts as contested is different in general
    if 'general' in filename:
        num_running = df_office.pivot_table(columns=['For the Office Of...'],
                                            aggfunc='size')
    
    # Loop through each party and specify if it is contested
    #...obviously this is only for primary
    for party in num_running.index:
        
        if num_running[party] < 2:
            if 'primary' in filename:
                df_all['Contested'][(df_all[office_col] == office)
                                    & (df_all['Party'] == party)] = 'no'

            if 'general' in filename:
                df_all['Contested'][(df_all[office_col] == office)] = 'no'      

In [48]:
# Split into contested and uncontested
df_contested = df_all[df_all['Contested'] == 'yes']
df_uncontested = df_all[df_all['Contested'] == 'no']

In [50]:
df_uncontested

,For the Office Of...,Ballot Name(s),Party,Email,Phone,Contested
26,State Senator District 3,Lynn Evans,Republican,evansforiowa@gmail.com,712-261-2726,no
28,State Senator District 4,Tim Kraayenbrink,Republican,tim.kbrink@gmail.com,515-408-4770,no
30,State Senator District 5,Dave Rowley,Republican,Daverowleysenate@gmail.com,712-330-5596,no
35,State Senator District 7,Kevin Alons,Republican,kevin@alonsforsenate.com,712-389-7016,no
46,State Senator District 15,Tony Bisignano,Democratic,biz.tony@gmail.com,515-371-5990,no
69,State Senator District 31,Bill Dotzler,Democratic,bdotzler@athenet.net,319-240-5594,no
86,State Senator District 43,Zach Wahls,Democratic,zach@zachwahlsforiowa.com,319-318-2094,no
96,State Senator District 49,Cindy Winckler,Democratic,cindywinckler@gmail.com,563-342-7927,no
98,State Representative District 1,J.D. Scholten,Democratic,jd@scholtenforiowa.com,612-242-0358,no
101,State Representative District 3,Thomas Jeneary,Republican,jeneary@premieronline.net,712-539-1275,no


In [44]:
# Write lists to different excel sheets
with pd.ExcelWriter('iowa-candidates-general-2022-test.xlsx') as writer:

    df_contested.to_excel(writer, sheet_name='contested',
                          index=False)
    df_uncontested.to_excel(writer, sheet_name='uncontested',
                            index=False)